<img src="Archivos/miad4.png" width=800x>

## Interactividad usando `panel`

En este taller podrás poner en práctica los conceptos aprendidos acerca del uso del paquete `panel`. Usarás `panel` con el fin de crear dashboards como herramienta de apoyo al momento de resolver problemas específicos en un contexto de Analytics.

Instrucciones y condiciones generales: Esta es una actividad calificada y, por lo tanto, debe ser resuelta individualmente.

### Al desarrollar este taller podrás verificar tu progreso en:

1. Programar la funcionalidad de un dashboard en `panel`.
2. Agregar gráficos y widgets para hacer visualizaciones agradables en `panel`.
3. Personalizar dashobards para generar distintas visualizaciones interactivas en `panel`.

In [1]:
import panel as pn
import plotly.graph_objects as go
import pandas as pd
pn.extension('plotly')

## Pregunta 1

Con la base de datos *COVID-19.csv* crea un dashboard en `panel` que:

1. Permita crear un diagrama de barras horizontal que muestre los casos de COVID-19 de acuerdo a la selección de los siguentes criterios:<br />
    a. Casos o muertes.<br />
    b. Periodo: Histórico, Últimos 7 días y Últimas 24 horas.<br />
    Pista: utilice un control de tipo `RadioButtonGroup` para cada uno de los dos criterios.<br />
2. Contenga un diagrama de barras horizontal de las muertes por millón de personas.
3. Permita visualizar a tráves de un diagrama de barras horizontal los casos en cada una de las regiones de la Organización Munidal de la Salud.

Nota: a excpeción de las regiones, muestre los 10 mayores valores en cada gráfico.

| <center>Variable</center> |<center>Descripción</center> | 
| :-- | :-- | 
| Name | País |
| WHO Region | Región de la OMS |
| Cases - cumulative total | Total de casos hasta el momento |
| Cases - cumulative total per 1 million population | Número de casos acumulado por millón de personas |
| Cases - newly reported in last 7 days | Número de casos nuevos en los últimos 7 días |
| Cases - newly reported in last 24 hours | Número de casos nuevos en las últimas 24 horas |
| Deaths - cumulative total | Total de muertes hasta el momento |
| Deaths - cumulative total per 1 million population | Número de muertes acumulado por millón de personas |
| Deaths - newly reported in last 7 days | Número de muertes nuevas en los últimos 7 días |
| Deaths - newly reported in last 24 hours | Número de muertes nuevas en las últimas 24 horas |

In [14]:
# YOUR CODE HERE
# Transformación base de datos para poder filtrar 
df = pd.read_csv('Archivos/COVID-19.csv')
df_unpvt = df.iloc[:, :10].melt(id_vars=['Name', 'WHO Region'], var_name = 'Clase_Casos', value_name = 'No')

def Muertes (x,var, var2):
    if x[var] == 'Cases - cumulative total'or x[var] =='Cases - cumulative total per 1 million population' or x[var] == 'Cases - newly reported in last 7 days'or x[var] == 'Cases - newly reported in last 24 hours':
        x[var2] ='Cases'
    else:
        x[var2] ='Deaths'
    return x 

def Periodo (x,var, var2):
    if x[var] == 'Cases - cumulative total'or x[var] =='Deaths - cumulative total':
        x[var2] ='Historico'
    elif x[var] == 'Cases - newly reported in last 7 days' or x[var] =='Deaths - newly reported in last 7 days':
        x[var2] ='7_days'
    elif x[var] == 'Cases - newly reported in last 24 hours' or x[var] =='Deaths - newly reported in last 24 hours':
        x[var2] ='24_hs'
    else: 
        x[var2] ='total_per_1_m'
    return x 

df_unpvt=df_unpvt.apply(lambda x: Muertes(x, 'Clase_Casos', 'Categoría'), axis = 1)
df_unpvt=df_unpvt.apply(lambda x: Periodo(x, 'Clase_Casos', 'Historial'), axis = 1)

df_unpvt.head(15)

,Name,WHO Region,Clase_Casos,No,Categoría,Historial
0,United States of America,Americas,Cases - cumulative total,26547977.0,Cases,Historico
1,India,South-East Asia,Cases - cumulative total,10826363.0,Cases,Historico
2,Brazil,Americas,Cases - cumulative total,9447165.0,Cases,Historico
3,Russian Federation,Europe,Cases - cumulative total,3967281.0,Cases,Historico
4,The United Kingdom,Europe,Cases - cumulative total,3929839.0,Cases,Historico
5,France,Europe,Cases - cumulative total,3262505.0,Cases,Historico
6,Spain,Europe,Cases - cumulative total,2913425.0,Cases,Historico
7,Italy,Europe,Cases - cumulative total,2625098.0,Cases,Historico
8,Turkey,Europe,Cases - cumulative total,2524786.0,Cases,Historico
9,Germany,Europe,Cases - cumulative total,2284010.0,Cases,Historico


In [43]:
# YOUR CODE HERE
Historial=df_unpvt["Historial"].unique().tolist() 

del Historial[1]

radio_group = pn.widgets.RadioButtonGroup(
    name='Radio Button Group', options= df_unpvt['Categoría'].unique().tolist() , button_type='success')

radio_group2 = pn.widgets.RadioButtonGroup(
    name='Radio Button Group', options= Historial , button_type='success')

In [48]:

#Punto 1
@pn.depends(radio_group.param.value, radio_group2.param.value) 
def interaccion1(radio_group, radio_group2): 

    # Cargamos los datos.
    df = df_unpvt
    # Creamos un filtro de datos que utiliza valores de la lista desplegable.
 
    # Filtramos el DataFrame.
    filtro = (df['Categoría'] == radio_group) & (df['Historial'] == radio_group2) 
    df = df.loc[filtro]
    # Agrupamos los datos por estación de llegada para poder contrarlos.
    df=df.groupby('Name').sum().reset_index()
    # Ordenamos los datos de mayor a menor.
    df.sort_values(by=['No'], inplace=True, ascending=False)
    
    

    
    # Definimos y retornamos la figura.
    fig = go.Figure(go.Bar(
                            x=df['No'],
                            y=df["Name"],
                            text=df['No'],
                            textposition='outside',                                # Posición de la etiqueta de datos. En este caso a la derecha.
                            texttemplate='%{text}',                                # Formato de texto.
                            marker_color='black',                      # Color de las barras.
                            orientation='h'                                        # Orientación de las barras. En este caso horizontal.
                          )
                   )
    
                    
    fig.update_layout(
                        barmode='stack',                                           # Tipo de barras. En este caso apiliadas.
                        yaxis_title="Paises",
                        xaxis_title="# " + radio_group,
                        title= str(radio_group) +" "+ radio_group2,
                        xaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje x.
                        yaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje y.
                        plot_bgcolor='white',                                      # Modificamos el color del fondo del gráfico.
                        showlegend=False,                                          # Ocultamos la leyenda.
                        yaxis_range=[20.5, -10]                                      # Mostramos los 10 primeros datos.
                     )

    
    return fig

In [49]:
#Punto2
@pn.depends(radio_group.param.value) 
def interaccion2(radio_group): 

    # Cargamos los datos.
    df = df_unpvt
    # Creamos un filtro de datos que utiliza valores de la lista desplegable.
 
    # Filtramos el DataFrame.
    filtro = (df['Categoría'] == radio_group) & (df['Historial'] == "total_per_1_m") 
    df = df.loc[filtro]
    # Agrupamos los datos por estación de llegada para poder contrarlos.
    df=df.groupby('Name').sum().reset_index()
    # Ordenamos los datos de mayor a menor.
    df.sort_values(by=['No'], inplace=True, ascending=False)
    
    

    
    # Definimos y retornamos la figura.
    fig = go.Figure(go.Bar(
                            x=df['No'],
                            y=df["Name"],
                            text=df['No'],
                            textposition='outside',                                # Posición de la etiqueta de datos. En este caso a la derecha.
                            texttemplate='%{text}',                                # Formato de texto.
                            marker_color='Black',                      # Color de las barras.
                            orientation='h'                                        # Orientación de las barras. En este caso horizontal.
                          )
                   )
    
                    
    fig.update_layout(
                        barmode='stack',                                           # Tipo de barras. En este caso apiliadas.
                        yaxis_title="Paises",
                        xaxis_title="# " + radio_group +" "+ "Número de muertes acumulado por MM de personas",
                        title= str(radio_group) + " " +"Número de muertes acumulado por MM de personas", 
                        xaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje x.
                        yaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje y.
                        plot_bgcolor='white',                                      # Modificamos el color del fondo del gráfico.
                        showlegend=False,                                          # Ocultamos la leyenda.
                        yaxis_range=[20.5, -10]                                     # Mostramos los 10 primeros datos.
                     )
    
    return fig

In [50]:
#Punto3

@pn.depends(radio_group.param.value, radio_group2.param.value) 
def interaccion3(radio_group, radio_group2): 

    # Cargamos los datos.
    df = df_unpvt
    # Creamos un filtro de datos que utiliza valores de la lista desplegable.
 
    # Filtramos el DataFrame.
    filtro = (df['Categoría'] == radio_group) & (df['Historial'] == radio_group2) 
    df = df.loc[filtro]
    # Agrupamos los datos por estación de llegada para poder contrarlos.
    df=df.groupby('WHO Region').sum().reset_index()
    # Ordenamos los datos de mayor a menor.
    df.sort_values(by=['No'], inplace=True, ascending=False)
    
    

    
    # Definimos y retornamos la figura.
    fig = go.Figure(go.Bar(
                            x=df['No'],
                            y=df["WHO Region"],
                            text=df['No'],
                            textposition='outside',                                # Posición de la etiqueta de datos. En este caso a la derecha.
                            texttemplate='%{text}',                                # Formato de texto.
                            marker_color='black',                      # Color de las barras.
                            orientation='h'                                        # Orientación de las barras. En este caso horizontal.
                          )
                   )
    
                    
    fig.update_layout(
                        barmode='stack',                                           # Tipo de barras. En este caso apiliadas.
                        yaxis_title="Casos por Regiones OMS",
                        xaxis_title="# " + radio_group,
                        title= str(radio_group) +" "+ radio_group2,
                        xaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje x.
                        yaxis=dict(showgrid=False),                                # Ocultamos la cuadrícula de eje y.
                        plot_bgcolor='white',                                      # Modificamos el color del fondo del gráfico.
                        showlegend=False,                                          # Ocultamos la leyenda.
                        
                     )

    
    return fig

In [51]:
#Final y visualizacion
Tablero_Final = pn.Column(pn.Row("",radio_group, radio_group2), interaccion1, interaccion2, interaccion3)
Tablero_Final.servable()

Column
    [0] Row
        [0] Markdown(str)
        [1] RadioButtonGroup(button_type='success', name='Radio Button Group', options=['Cases', 'Deaths'], value='Cases')
        [2] RadioButtonGroup(button_type='success', name='Radio Button Group', options=['Historico', ...], value='Historico')
    [1] ParamFunction(function)
    [2] ParamFunction(function)
    [3] ParamFunction(function)

In [53]:
#Navegador
Tablero_Final.show()

Launching server at http://localhost:60673


## Créditos

**Autor(es)**: Juan David Reyes Jaimes, Diego Alejandro Cely Gomez

**Última actualización:** 26/08/2021